In [1]:
import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [11]:
import geopandas as gpd
import shapely
from geopandas.tools import sjoin

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/geopandas/_compat.py:110: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.0-CAPI-1.16.2). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


In [7]:
#import raw csv
raw_df = pd.read_csv("data/census_tract/financial_char/data_financial.csv", header=1)

In [17]:
raw_df = raw_df.set_index('id')

In [12]:
raw_gdf = gpd.read_file('data/census_tract/shape/cb_2018_25_tract_500k.shp')

In [16]:
raw_gdf = raw_gdf.set_index('AFFGEOID')

In [20]:
raw_df.shape

(204, 553)

In [21]:
raw_gdf.shape

(1475, 9)

In [173]:
household_income_units = raw_df.filter(like='Occupied housing units!!Occupied housing units', axis=1).filter(like='HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2018 INFLATION-ADJUSTED DOLLARS)', axis=1).filter(like='Estimate', axis=1)

In [174]:
household_income_units.shape

(204, 12)

In [175]:
household_income_units.head()

,"Estimate!!Occupied housing units!!Occupied housing units!!HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2018 INFLATION-ADJUSTED DOLLARS)!!Less than $5,000","Estimate!!Occupied housing units!!Occupied housing units!!HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2018 INFLATION-ADJUSTED DOLLARS)!!$5,000 to $9,999","Estimate!!Occupied housing units!!Occupied housing units!!HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2018 INFLATION-ADJUSTED DOLLARS)!!$10,000 to $14,999","Estimate!!Occupied housing units!!Occupied housing units!!HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2018 INFLATION-ADJUSTED DOLLARS)!!$15,000 to $19,999","Estimate!!Occupied housing units!!Occupied housing units!!HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2018 INFLATION-ADJUSTED DOLLARS)!!$20,000 to $24,999","Estimate!!Occupied housing units!!Occupied housing units!!HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2018 INFLATION-ADJUSTED DOLLARS)!!$25,000 to $34,999","Estimate!!Occupied housing units!!Occupied housing units!!HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2018 INFLATION-ADJUSTED DOLLARS)!!$35,000 to $49,999","Estimate!!Occupied housing units!!Occupied housing units!!HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2018 INFLATION-ADJUSTED DOLLARS)!!$50,000 to $74,999","Estimate!!Occupied housing units!!Occupied housing units!!HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2018 INFLATION-ADJUSTED DOLLARS)!!$75,000 to $99,999","Estimate!!Occupied housing units!!Occupied housing units!!HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2018 INFLATION-ADJUSTED DOLLARS)!!$100,000 to $149,999","Estimate!!Occupied housing units!!Occupied housing units!!HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2018 INFLATION-ADJUSTED DOLLARS)!!$150,000 or more",Estimate!!Occupied housing units!!Occupied housing units!!HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2018 INFLATION-ADJUSTED DOLLARS)!!Median household income (dollars)
id,,,,,,,,,,,,
1400000US25025000100,36,8,91,91,133,128,140,238,215,388,314,79597
1400000US25025000201,24,41,0,30,63,105,93,249,254,318,354,88424
1400000US25025000202,29,53,71,12,47,132,117,220,192,296,333,83000
1400000US25025000301,0,11,53,0,41,40,114,111,161,226,411,106897
1400000US25025000302,36,57,26,0,52,52,124,121,150,440,231,100883


In [176]:
def clean_title(s):
    while('!!' in s):
        index = s.index('!!')
        s = s[index+2:]
    if('𝑡𝑜' in s):
        index = s.index('𝑡𝑜')
        s = s[:index] + ' to' + s[index+2:]
    return s.strip()

In [177]:
clean_title('Estimate!!Occupied housing units!!Occupied housing units!!HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2018 INFLATION-ADJUSTED DOLLARS)!! 5,000𝑡𝑜 9,999')

'5,000 to 9,999'

In [178]:
income_labels = ['<5000', '5k~10k', '10k~15k', '15k~20k', '20k~25k', '25k~35k', '35k~50k', '50k~75k', '75~100k', '100k~150k', '>150k', 'medHHIncome']

In [181]:
household_income_units = household_income_units.set_axis(income_labels, axis=1)

In [186]:
household_income_total = household_income_units.loc[household_income_units['medHHIncome'] != '-'][['medHHIncome']].apply(pd.to_numeric)

In [182]:
household_income_gdf = pd.merge(raw_gdf, household_income_units, left_index=True, right_index=True, how="inner")

In [187]:
household_income_total = pd.merge(raw_gdf, household_income_total, left_index=True, right_index=True, how="inner")

In [188]:
household_income_total.head()

,STATEFP,COUNTYFP,TRACTCE,GEOID,NAME,LSAD,ALAND,AWATER,geometry,medHHIncome
1400000US25025050500,25,025,050500,25025050500,505,CT,111868,0,"POLYGON ((-71.03699 42.36929, -71.03392 42.371...",61029
1400000US25025060101,25,025,060101,25025060101,601.01,CT,295623,0,"POLYGON ((-71.03128 42.33483, -71.02923 42.335...",119107
1400000US25025060400,25,025,060400,25025060400,604,CT,399279,0,"POLYGON ((-71.04751 42.33460, -71.04569 42.335...",123967
1400000US25025061101,25,025,061101,25025061101,611.01,CT,165282,0,"POLYGON ((-71.05693 42.32855, -71.05262 42.328...",12759
1400000US25025061200,25,025,061200,25025061200,612,CT,1835771,69804,"POLYGON ((-71.06553 42.33603, -71.06239 42.342...",130625


In [148]:
household_income_gdf.to_file('data/household_income_gdf.shp')

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  """Entry point for launching an IPython kernel.


In [189]:
household_income_total.to_file('data/house_income_total.shp')

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  """Entry point for launching an IPython kernel.


In [30]:
clean_gdf = pd.merge(raw_gdf, raw_df, left_index=True, right_index=True, how="inner")

In [33]:
clean_gdf.shape

(204, 562)

In [192]:
raw_education_df = pd.read_csv("data/educational_attainment/data.csv", header=1)

In [238]:
raw_education_df = raw_education_df.set_index('id')

In [239]:
raw_education_df.head()

,Geographic Area Name,Estimate!!Total!!Population 18 to 24 years,Margin of Error!!Total MOE!!Population 18 to 24 years,Estimate!!Total!!Population 18 to 24 years!!Less than high school graduate,Margin of Error!!Total MOE!!Population 18 to 24 years!!Less than high school graduate,Estimate!!Total!!Population 18 to 24 years!!High school graduate (includes equivalency),Margin of Error!!Total MOE!!Population 18 to 24 years!!High school graduate (includes equivalency),Estimate!!Total!!Population 18 to 24 years!!Some college or associate's degree,Margin of Error!!Total MOE!!Population 18 to 24 years!!Some college or associate's degree,Estimate!!Total!!Population 18 to 24 years!!Bachelor's degree or higher,...,Estimate!!Percent Female!!MEDIAN EARNINGS IN THE PAST 12 MONTHS (IN 2018 INFLATION-ADJUSTED DOLLARS)!!Population 25 years and over with earnings!!Less than high school graduate,Margin of Error!!Percent Female MOE!!MEDIAN EARNINGS IN THE PAST 12 MONTHS (IN 2018 INFLATION-ADJUSTED DOLLARS)!!Population 25 years and over with earnings!!Less than high school graduate,Estimate!!Percent Female!!MEDIAN EARNINGS IN THE PAST 12 MONTHS (IN 2018 INFLATION-ADJUSTED DOLLARS)!!Population 25 years and over with earnings!!High school graduate (includes equivalency),Margin of Error!!Percent Female MOE!!MEDIAN EARNINGS IN THE PAST 12 MONTHS (IN 2018 INFLATION-ADJUSTED DOLLARS)!!Population 25 years and over with earnings!!High school graduate (includes equivalency),Estimate!!Percent Female!!MEDIAN EARNINGS IN THE PAST 12 MONTHS (IN 2018 INFLATION-ADJUSTED DOLLARS)!!Population 25 years and over with earnings!!Some college or associate's degree,Margin of Error!!Percent Female MOE!!MEDIAN EARNINGS IN THE PAST 12 MONTHS (IN 2018 INFLATION-ADJUSTED DOLLARS)!!Population 25 years and over with earnings!!Some college or associate's degree,Estimate!!Percent Female!!MEDIAN EARNINGS IN THE PAST 12 MONTHS (IN 2018 INFLATION-ADJUSTED DOLLARS)!!Population 25 years and over with earnings!!Bachelor's degree,Margin of Error!!Percent Female MOE!!MEDIAN EARNINGS IN THE PAST 12 MONTHS (IN 2018 INFLATION-ADJUSTED DOLLARS)!!Population 25 years and over with earnings!!Bachelor's degree,Estimate!!Percent Female!!MEDIAN EARNINGS IN THE PAST 12 MONTHS (IN 2018 INFLATION-ADJUSTED DOLLARS)!!Population 25 years and over with earnings!!Graduate or professional degree,Margin of Error!!Percent Female MOE!!MEDIAN EARNINGS IN THE PAST 12 MONTHS (IN 2018 INFLATION-ADJUSTED DOLLARS)!!Population 25 years and over with earnings!!Graduate or professional degree
id,,,,,,,,,,,,,,,,,,,,,
1400000US25025000100,"Census Tract 1, Suffolk County, Massachusetts",570,120,82,86,92,56,131,62,265,...,(X),(X),(X),(X),(X),(X),(X),(X),(X),(X)
1400000US25025000201,"Census Tract 2.01, Suffolk County, Massachusetts",516,159,9,15,23,23,136,116,348,...,(X),(X),(X),(X),(X),(X),(X),(X),(X),(X)
1400000US25025000202,"Census Tract 2.02, Suffolk County, Massachusetts",494,165,9,16,60,38,147,83,278,...,(X),(X),(X),(X),(X),(X),(X),(X),(X),(X)
1400000US25025000301,"Census Tract 3.01, Suffolk County, Massachusetts",437,148,50,66,36,56,74,45,277,...,(X),(X),(X),(X),(X),(X),(X),(X),(X),(X)
1400000US25025000302,"Census Tract 3.02, Suffolk County, Massachusetts",353,119,24,36,1,2,159,106,169,...,(X),(X),(X),(X),(X),(X),(X),(X),(X),(X)


In [252]:
educational_attainment_units = raw_education_df.filter(like='Estimate!!Total!!P', axis=1).iloc[:, :15]

In [253]:
educational_attainment_units.head()

,Estimate!!Total!!Population 18 to 24 years,Estimate!!Total!!Population 18 to 24 years!!Less than high school graduate,Estimate!!Total!!Population 18 to 24 years!!High school graduate (includes equivalency),Estimate!!Total!!Population 18 to 24 years!!Some college or associate's degree,Estimate!!Total!!Population 18 to 24 years!!Bachelor's degree or higher,Estimate!!Total!!Population 25 years and over,Estimate!!Total!!Population 25 years and over!!Less than 9th grade,"Estimate!!Total!!Population 25 years and over!!9th to 12th grade, no diploma",Estimate!!Total!!Population 25 years and over!!High school graduate (includes equivalency),"Estimate!!Total!!Population 25 years and over!!Some college, no degree",Estimate!!Total!!Population 25 years and over!!Associate's degree,Estimate!!Total!!Population 25 years and over!!Bachelor's degree,Estimate!!Total!!Population 25 years and over!!Graduate or professional degree,Estimate!!Total!!Population 25 years and over!!High school graduate or higher,Estimate!!Total!!Population 25 years and over!!Bachelor's degree or higher
id,,,,,,,,,,,,,,,
1400000US25025000100,570,82,92,131,265,3969,396,111,1018,303,178,1193,770,3462,1963
1400000US25025000201,516,9,23,136,348,3100,107,45,322,342,100,1498,686,2948,2184
1400000US25025000202,494,9,60,147,278,2839,157,213,394,399,139,967,570,2469,1537
1400000US25025000301,437,50,36,74,277,2182,77,128,463,156,47,905,406,1977,1311
1400000US25025000302,353,24,1,159,169,2615,103,114,371,296,62,999,670,2398,1669


In [254]:
drop_labels = ['Estimate!!Total!!Population 18 to 24 years', 'Estimate!!Total!!Population 25 years and over', 'Estimate!!Total!!Population 25 years and over!!High school graduate or higher', 'Estimate!!Total!!Population 25 years and over!!Bachelor\'s degree or higher']

In [255]:
educational_attainment_clean = educational_attainment_units.drop(columns=drop_labels)

In [258]:
print(educational_attainment_clean.shape)
educational_attainment_clean.head()

(204, 11)


,Estimate!!Total!!Population 18 to 24 years!!Less than high school graduate,Estimate!!Total!!Population 18 to 24 years!!High school graduate (includes equivalency),Estimate!!Total!!Population 18 to 24 years!!Some college or associate's degree,Estimate!!Total!!Population 18 to 24 years!!Bachelor's degree or higher,Estimate!!Total!!Population 25 years and over!!Less than 9th grade,"Estimate!!Total!!Population 25 years and over!!9th to 12th grade, no diploma",Estimate!!Total!!Population 25 years and over!!High school graduate (includes equivalency),"Estimate!!Total!!Population 25 years and over!!Some college, no degree",Estimate!!Total!!Population 25 years and over!!Associate's degree,Estimate!!Total!!Population 25 years and over!!Bachelor's degree,Estimate!!Total!!Population 25 years and over!!Graduate or professional degree
id,,,,,,,,,,,
1400000US25025000100,82,92,131,265,396,111,1018,303,178,1193,770
1400000US25025000201,9,23,136,348,107,45,322,342,100,1498,686
1400000US25025000202,9,60,147,278,157,213,394,399,139,967,570
1400000US25025000301,50,36,74,277,77,128,463,156,47,905,406
1400000US25025000302,24,1,159,169,103,114,371,296,62,999,670


In [274]:
under_hs = pd.concat([educational_attainment_clean.iloc[:, 0], educational_attainment_clean.iloc[:, 4:6]], axis=1).sum(axis=1)
under_hs.name = 'NoDegree'
under_hs

id
1400000US25025000100    589
1400000US25025000201    161
1400000US25025000202    379
1400000US25025000301    255
1400000US25025000302    241
                       ... 
1400000US25025981502      0
1400000US25025981600      0
1400000US25025981700      0
1400000US25025981800      0
1400000US25025990101      0
Name: NoDegree, Length: 204, dtype: int64

In [275]:
hs = pd.concat([educational_attainment_clean.iloc[:, 2:4], educational_attainment_clean.iloc[:, 6:8]], axis=1).sum(axis=1)
hs.name = 'HSDegree'
hs

id
1400000US25025000100    1717
1400000US25025000201    1148
1400000US25025000202    1218
1400000US25025000301     970
1400000US25025000302     995
                        ... 
1400000US25025981502       0
1400000US25025981600       0
1400000US25025981700       0
1400000US25025981800       0
1400000US25025990101      49
Name: HSDegree, Length: 204, dtype: int64

In [276]:
college = pd.concat([educational_attainment_clean.iloc[:, 3], educational_attainment_clean.iloc[:, 8:]], axis=1).sum(axis=1)
college.name = 'CollegeDegree'
college

id
1400000US25025000100    2406
1400000US25025000201    2632
1400000US25025000202    1954
1400000US25025000301    1635
1400000US25025000302    1900
                        ... 
1400000US25025981502       0
1400000US25025981600       0
1400000US25025981700       0
1400000US25025981800      21
1400000US25025990101       0
Name: CollegeDegree, Length: 204, dtype: int64

In [277]:
educational_attain_pdf = pd.concat([under_hs, hs, college], axis=1)

In [278]:
educational_attain_pdf.head()

,NoDegree,HSDegree,CollegeDegree
id,,,
1400000US25025000100,589,1717,2406
1400000US25025000201,161,1148,2632
1400000US25025000202,379,1218,1954
1400000US25025000301,255,970,1635
1400000US25025000302,241,995,1900


In [282]:
educational_attain_pdf['Degree'] = educational_attain_pdf.idxmax(axis=1)

In [283]:
educational_attain_gdf = pd.merge(raw_gdf, educational_attain_pdf, left_index=True, right_index=True, how="inner")

In [284]:
educational_attain_gdf.head()

,STATEFP,COUNTYFP,TRACTCE,GEOID,NAME,LSAD,ALAND,AWATER,geometry,NoDegree,HSDegree,CollegeDegree,Degree
1400000US25025050500,25,025,050500,25025050500,505,CT,111868,0,"POLYGON ((-71.03699 42.36929, -71.03392 42.371...",520,592,694,CollegeDegree
1400000US25025060101,25,025,060101,25025060101,601.01,CT,295623,0,"POLYGON ((-71.03128 42.33483, -71.02923 42.335...",114,781,2279,CollegeDegree
1400000US25025060400,25,025,060400,25025060400,604,CT,399279,0,"POLYGON ((-71.04751 42.33460, -71.04569 42.335...",188,1300,4052,CollegeDegree
1400000US25025061101,25,025,061101,25025061101,611.01,CT,165282,0,"POLYGON ((-71.05693 42.32855, -71.05262 42.328...",498,690,319,HSDegree
1400000US25025061200,25,025,061200,25025061200,612,CT,1835771,69804,"POLYGON ((-71.06553 42.33603, -71.06239 42.342...",204,999,3104,CollegeDegree


In [286]:
educational_attain_gdf.to_file('data/educational_attain_gdf.shp')

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  """Entry point for launching an IPython kernel.
